In [329]:
import json
import csv
import logging
import itertools
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 
from pandas import read_csv
from functools import reduce
from collections import defaultdict

In [340]:
etl = 'test2newaqm'

if etl == 'pototan2newaqm':
    resident_file = 'file/parsed_output/Pototan/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Pototan/output.csv'
    parsed_output_json_file = 'file/parsed_output/Pototan/output.json'
    
elif etl == 'cambodia2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Cambodia/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Cambodia/output.csv'
    parsed_output_json_file = 'file/parsed_output/Cambodia/output.json'
    
elif etl == 'isabela2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Isabela/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Isabela/output.csv'
    parsed_output_json_file = 'file/parsed_output/Isabela/output.json'
    
elif etl == 'guimbal2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Guimbal/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Guimbal/output.csv'
    parsed_output_json_file = 'file/parsed_output/Guimbal/output.json'
    
elif etl == 'pototan2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Pototan/resident.raw.csv'
    parsed_output_csv_file = 'file/parsed_output/Pototan/output.csv'
    parsed_output_json_file = 'file/parsed_output/Pototan/output.json'

elif etl == 'cuartero2newaqm':
    #kobo to elasticsearch
    resident_file = 'file/parsed_output/Cuartero/resident.csv'
    parsed_output_csv_file = 'file/parsed_output/Cuartero/output.csv'
    parsed_output_json_file = 'file/parsed_output/Cuartero/output.json'

elif etl == 'test2newaqm':
    datelog_dir = 'couchbase-curis-2019-06-21'
    processed_dir    = 'data/processed/' + datelog_dir + '/'
    tmp_dir    = 'data/processed/' + datelog_dir + '/tmp/'
    output_dir = 'data/processed/' + datelog_dir + '/'
    merged_dir = 'data/merged/' + datelog_dir + '/'
    
    resident_file           = 'data/merged/couchbase-curis-2019-06-21-cuartero/health_information.csv'
    parsed_output_csv_file  = 'data/merged/couchbase-curis-2019-06-21-cuartero/health_information.tmp.csv'
    parsed_output_json_file = 'data/merged/couchbase-curis-2019-06-21-cuartero/health_information.output.json'

In [331]:
resident_df = pd.DataFrame()
resident_df = pd.read_csv(resident_file, encoding = "ISO-8859-1") 
#hc_df.reset_index(level=0, inplace=True)
#resident_df = resident_df.replace(np.nan,'',regex=True)
resident_df.head(3)

,_id,_index_map,answers.allergies,answers.bp1Diastole,answers.bp1Systole,answers.bp2Diastole,answers.bp2Systole,answers.rhesusType,answers.bloodType,answers.weeklyExercise,answers.smokingHabit,answers.consentBloodTest,answers.dailyFiberIntake,dateCreated,dateUpdated,formId,formName,type,profileId,id
0,a0a8cfc1-114a-4467-a299-156cc341cd7a,NaN,NaN,NaN,NaN,NaN,NaN,empty,I don't know,0 to 1 hour,Never,N,NaN,2019-06-26T18:53:55.460+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form,0003ff38-28fb-4005-9437-d276cbb9da4d,a0a8cfc1-114a-4467-a299-156cc341cd7a
1,4a58e7ec-458c-416e-bbf5-107ec2507144,NaN,NaN,NaN,NaN,NaN,NaN,empty,I don't know,0 to 1 hour,Never,N,NaN,2019-06-26T18:53:55.460+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form,0003ff38-28fb-4005-9437-d276cbb9da4d,4a58e7ec-458c-416e-bbf5-107ec2507144
2,43c46680-9a25-4f5f-9c3c-b1b52546b6e1,NaN,NaN,NaN,NaN,NaN,NaN,empty,I don't know,0 to 1 hour,Never,N,NaN,2019-06-26T18:53:55.460+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form,0003ff38-28fb-4005-9437-d276cbb9da4d,43c46680-9a25-4f5f-9c3c-b1b52546b6e1


In [332]:
resident_df.T

,0,1,2,3,4,5,6,7,8,9,...,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919
_id,a0a8cfc1-114a-4467-a299-156cc341cd7a,4a58e7ec-458c-416e-bbf5-107ec2507144,43c46680-9a25-4f5f-9c3c-b1b52546b6e1,c9cc19b6-9d35-443e-b7ae-fc9ef6ccdba4,f25ac573-d234-42eb-bffa-132a4768e5e0,703f9d99-6b1f-4013-8b13-285a1efe3f9f,1ca3c2c3-ef38-43ab-840e-67cc35dc97e4,ce7ee55d-ae3d-4605-bd5e-4ba625958602,fdcf73cd-9cc2-4085-8a1c-2f075b646584,3e99059f-4158-446b-9f75-4be1b0901e11,...,2f720881-4116-4613-a119-b8ee0d0c7fe9,11447e2d-a303-4b58-8785-9ceac12e5071,c8d0f31e-5703-4695-b65a-dadb6aab903c,aa8807ad-490c-4bb1-a12e-193627f88dc7,9ca63739-0080-423d-b20f-3685f6c87174,32625767-4ca0-4069-b7c6-ca74cda0c53c,6f340826-cdd9-47b1-8f9a-ffbf3c1535fe,14ac7419-22a2-4981-a68a-c251cc85861d,be209c2f-99ee-4605-a31b-9cc87eb8d866,3ac835bb-7ce1-4358-ba07-45a9152efd60
_index_map,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
answers.allergies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,...,NaN,none,none,none,NaN,no,no,no,no,NaN
answers.bp1Diastole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,80,...,70,NaN,NaN,NaN,NaN,80,NaN,70,NaN,NaN
answers.bp1Systole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110,110,...,110,NaN,NaN,NaN,NaN,120,NaN,120,NaN,NaN
answers.bp2Diastole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,NaN,NaN
answers.bp2Systole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,NaN,NaN
answers.rhesusType,empty,empty,empty,Wala ko kabalo,Wala ko kabalo,Wala ko kabalo,empty,empty,empty,Wala ko kabalo,...,empty,I don't know,I don't know,I don't know,empty,Wala ko kabalo,Wala ko kabalo,Wala ko kabalo,I don't know,I don't know
answers.bloodType,I don't know,I don't know,I don't know,Wala ko kabalo,Wala ko kabalo,Wala ko kabalo,I don't know,I don't know,O,Wala ko kabalo,...,I don't know,I don't know,I don't know,I don't know,I don't know,Wala ko kabalo,Wala ko kabalo,Wala ko kabalo,I don't know,I don't know
answers.weeklyExercise,0 to 1 hour,0 to 1 hour,0 to 1 hour,NaN,NaN,NaN,0 to 1 hour,0 to 1 hour,0 to 1 hour,0 to 1 Oras,...,0 to 1 hour,0 to 1 hour,0 to 1 hour,0 to 1 hour,2 to 3 hours,0 to 1 Oras,Subra sa 5 ka oras,Subra sa 5 ka oras,0 to 1 hour,2 to 3 hours


In [333]:
def concat_values(items):
    _list = []
    _index_map = ''
    _id = ''
    
    for index ,value in items.iteritems():
        _new_item = ''
        
        ##TODO: Check unqiue identifer here
        if index == "_id":
        #if index == "id":
            _id = value
            #print(_index_map)
            _list.append(_id)
           
        elif index == "_index_map":
            _index_map = value
            
            _list.append(_index_map)
            #TODO: MUST BE A FUNCTION --> Parser of index_map
        else:
            if len(index) > 1 :
                
                _new_index = index.split(sep='.')
                
                
                if pd.isna(_index_map) :
                    _new_index[0] = _new_index[0]
                
                else:
                    
                    _new_index[0] = _new_index[0] + '|' + str(_index_map)
                    
                _edited_index = '.'.join(_new_index)
                _edited_index

                    #===========
                    #_new_item = index + ':"' + str(value) + '"'
                _new_item = _edited_index + ":'" + str(value) + "'"
                _list.append(_new_item)
            else:
                continue
            
    return _list    

In [334]:
new_residents_df = pd.DataFrame()
new_resident_df = resident_df.T.apply(concat_values)
new_resident_df = new_resident_df.drop(index='_index_map')
#TODO: Change idnetifier _id formid id accordingly
#new_resident_df = new_resident_df.drop(index='_id')
new_resident_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,17910,17911,17912,17913,17914,17915,17916,17917,17918,17919
_id,a0a8cfc1-114a-4467-a299-156cc341cd7a,4a58e7ec-458c-416e-bbf5-107ec2507144,43c46680-9a25-4f5f-9c3c-b1b52546b6e1,c9cc19b6-9d35-443e-b7ae-fc9ef6ccdba4,f25ac573-d234-42eb-bffa-132a4768e5e0,703f9d99-6b1f-4013-8b13-285a1efe3f9f,1ca3c2c3-ef38-43ab-840e-67cc35dc97e4,ce7ee55d-ae3d-4605-bd5e-4ba625958602,fdcf73cd-9cc2-4085-8a1c-2f075b646584,3e99059f-4158-446b-9f75-4be1b0901e11,...,2f720881-4116-4613-a119-b8ee0d0c7fe9,11447e2d-a303-4b58-8785-9ceac12e5071,c8d0f31e-5703-4695-b65a-dadb6aab903c,aa8807ad-490c-4bb1-a12e-193627f88dc7,9ca63739-0080-423d-b20f-3685f6c87174,32625767-4ca0-4069-b7c6-ca74cda0c53c,6f340826-cdd9-47b1-8f9a-ffbf3c1535fe,14ac7419-22a2-4981-a68a-c251cc85861d,be209c2f-99ee-4605-a31b-9cc87eb8d866,3ac835bb-7ce1-4358-ba07-45a9152efd60
answers.allergies,answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'nan',answers.allergies:'no',...,answers.allergies:'nan',answers.allergies:'none',answers.allergies:'none',answers.allergies:'none',answers.allergies:'nan',answers.allergies:'no',answers.allergies:'no',answers.allergies:'no',answers.allergies:'no',answers.allergies:'nan'
answers.bp1Diastole,answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'80.0',answers.bp1Diastole:'80.0',...,answers.bp1Diastole:'70.0',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan',answers.bp1Diastole:'80.0',answers.bp1Diastole:'nan',answers.bp1Diastole:'70.0',answers.bp1Diastole:'nan',answers.bp1Diastole:'nan'


In [341]:
def insert_concat(items):
    test_df = pd.DataFrame()
    _test_obj = []
    _test_id = ''
    for index ,value in items.iteritems():
        _kv = {}
        
        #TODO: change _id here in formid, id or id
        if index == '_id':
            #print(index)
            _test_id = value
            continue
        else:
            _kv["id"] = _test_id
            _kv["value"] = value
            
        _test_obj.append(_kv)
    return _test_obj

In [336]:
vertical_resident_df = []
vertical_resident_df = new_resident_df.apply(insert_concat)

flat = list(itertools.chain.from_iterable(vertical_resident_df))
flat_df = pd.DataFrame.from_dict(flat)
flat_df.head(3)

,id,value
0,a0a8cfc1-114a-4467-a299-156cc341cd7a,answers.allergies:'nan'
1,a0a8cfc1-114a-4467-a299-156cc341cd7a,answers.bp1Diastole:'nan'
2,a0a8cfc1-114a-4467-a299-156cc341cd7a,answers.bp1Systole:'nan'


## WRITE TO CSV FILE

In [337]:
#TODO: delete if exisitng
flat_df.to_csv(parsed_output_csv_file, encoding='utf-8', mode='w', index=False)

In [338]:
hc_df = pd.DataFrame()
hc_df = pd.read_csv(parsed_output_csv_file,header=None, dtype=str) 
hc_df.head(3)

,0,1
0,id,value
1,a0a8cfc1-114a-4467-a299-156cc341cd7a,answers.allergies:'nan'
2,a0a8cfc1-114a-4467-a299-156cc341cd7a,answers.bp1Diastole:'nan'


## TODO BUG: if _id is not in index[0]

In [339]:
def setValue(value, field, it):
    if isinstance(value, dict) or isinstance(value, list) and field not in it: # prevent override
            it[field] = value
    if not isinstance(value, dict) and not isinstance(value, list):
            it[field] = value

def findField(ds, k, keyVal, it):
    it = ds[k]
    if isinstance(it, list): # if array then find correct index
        index = int(keyVal[1])
        if len(it) > index:
            it = it[index]
    return it

def setAttribute(profile, lastKeys, field, value):
    it = {}
    for key in lastKeys:
        keyVal = key.split('|')
        k = keyVal[0]
        if not it:
            it = findField(profile, k, keyVal, it)
        else:
            it = findField(it, k, keyVal, it)

    if isinstance(it, list):
        index = int(lastKeys[-1].split('|')[1]) # get index from ['contactnumber/0']
        if len(it) > index: # find element and element attribute to set value
            setValue(value, field, it[index])
        else: # list is empty so create an element
            for i in range(len(it), index + 1, 1):
                it.append({})
            it[index][field] = value
    else:
        setValue(value, field, it)

def defineField(profile, field):
    keyVal = field.split('|')
    if len(keyVal) > 1:
        key = keyVal[0]
        if key not in profile:
            profile[key] = []
    elif field not in profile:
        profile[field] = {}

profiles = []

def properValue(value):
    proper = None
    try:
        proper = eval(value)
    except:
        proper = value
    return proper

# app starts here


print('parser running...')
with open(parsed_output_csv_file, encoding="utf-8") as csv_file:
#with open(resident_file, encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    profile = {}
    lastId = None
    firstProfile = True
    for row in csv_reader: # read the csv line by line
        if line_count > 0: # don't read columns
            # print(row[1].rsplit(':', 1))
            fieldValueRaw = row[1]
            fieldValue = fieldValueRaw.split(":", 1)
            rawFields = [x.strip() for x in fieldValue[0].split('.')]
            rawFields[-1] = rawFields[-1] + ':' + fieldValue[1]
            # stop updating profile when id changes
            # add profile and build it afterwards
            if lastId != row[0]: # new id means add and build profile
                if not firstProfile:
                    profile = {} # reset profile and build it
                profile['id'] = row[0]
                profiles.append(profile) # this profile will be built below
                lastId = row[0]
                firstProfile = False

            lastKeys = []
            for field in rawFields: # build the profile up
                keyVal = field.split(':', 1) # field and value format, we separate it
                if len(keyVal) == 1: # ['answers'] if field doesn't contain value we build it
                    if len(lastKeys) == 0:
                        defineField(profile, field)
                    else:
                        _keyVal = field.split('|')
                        if len(_keyVal) > 1:
                            setAttribute(profile, lastKeys, _keyVal[0], [])
                        else:
                            setAttribute(profile, lastKeys, field, {})
                    lastKeys.append(field)
                else: # ['type', 'mobile']
                    if len(lastKeys) == 0:
                        profile[keyVal[0]] = properValue(keyVal[1])
                    else:
                        setAttribute(profile, lastKeys, keyVal[0], properValue(keyVal[1]))
                    lastKeys.append(keyVal[0])
        line_count += 1

#os.makedirs('output', exist_ok=True)
#file = open(parsed_output_json_file, 'w')
#file.write(json.dumps(, indent=4, sort_keys=True))
#file.write(json.dumps(profiles))
#file.close()

#encoding = "ISO-8859-1"
with open(parsed_output_json_file, 'w', encoding="utf-8") as outfile:
#with open(parsed_output_json_file, 'w', encoding="ISO-8859-1") as outfile: #For KHM
    json.dump(profiles, outfile, indent=4, sort_keys=True, ensure_ascii=False)
    
print('parser completed...')

parser running...
parser completed...
